In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf

In [3]:
# 데이터 준비
codes = ['']

In [ ]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout
from spektral.layers import GCNConv
from spektral.data import Graph

# 여러 종목의 데이터 로드
tickers = ['AAPL', 'GOOGL', 'MSFT', 'AMZN', 'TSLA']
dataframes = []

for ticker in tickers:
    df = pd.read_csv(f'{ticker}_data.csv')  # 각 종목의 CSV 파일
    df['ticker'] = ticker  # 종목 이름 추가
    dataframes.append(df)

# 모든 종목 데이터 결합
all_data = pd.concat(dataframes)

# 데이터 전처리
scaler = MinMaxScaler()
all_data['scaled_price'] = scaler.fit_transform(all_data['stock_price'].values.reshape(-1, 1))

# 시계열 데이터 생성
def create_dataset(data, time_step=1):
    X, y = [], []
    for i in range(len(data) - time_step):
        X.append(data[i:(i + time_step), 0])
        y.append(data[i + time_step, 0])
    return np.array(X), np.array(y)

# 종목별로 데이터 준비
stock_data = {}
for ticker in tickers:
    ticker_data = all_data[all_data['ticker'] == ticker]['scaled_price'].values
    X, y = create_dataset(ticker_data.reshape(-1, 1), time_step=10)
    stock_data[ticker] = (X, y)

# GCN을 위한 그래프 구성
num_nodes = len(tickers)
edges = np.array([[0, 1], [1, 2], [2, 3], [3, 4], [4, 0]])  # 예시 엣지
node_features = np.random.rand(num_nodes, 3)  # 각 종목의 특성 (예: 가격, 거래량 등)

# 그래프 데이터 생성
graph = Graph(x=node_features, a=edges)

# 모델 정의
def build_model(input_shape, num_nodes):
    gcn_input = Input(shape=(num_nodes, input_shape[1]))  # num_nodes 사용
    x = GCNConv(32, activation='relu')([gcn_input, graph.a])
    x = Dropout(0.5)(x)

    lstm_input = Input(shape=(10, 1))  # 시계열 입력
    lstm_out = LSTM(50, return_sequences=True)(lstm_input)
    lstm_out = LSTM(50)(lstm_out)

    combined = Dense(32, activation='relu')([x, lstm_out])
    output = Dense(1)(combined)

    model = Model(inputs=[gcn_input, lstm_input], outputs=output)
    model.compile(optimizer='adam', loss='mean_squared_error')
    return model

# 모델 구축
model = build_model((num_nodes, node_features.shape[1]), num_nodes)

# 각 종목 데이터로 모델 훈련
for ticker in tickers:
    X, y = stock_data[ticker]
    model.fit([graph.x, X.reshape(-1, 10, 1)], y, epochs=50, batch_size=32)

# 예측
predictions = model.predict([graph.x, X.reshape(-1, 10, 1)])
predicted_prices = scaler.inverse_transform(predictions)

print(predicted_prices)
